In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import etree as et
import time
import random
import json
from unidecode import unidecode

In [2]:
base_genre_url = 'https://www.imdb.com/feature/genre/'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}

response = requests.get(base_genre_url, headers=header)
soup = BeautifulSoup(response.text, 'html.parser')

req_section = soup.find('a', {'href': '#movie'}).find_parent('section')

req_div = req_section.find('div', class_="ipc-chip-list--base ipc-chip-list ipc-chip-list--wrap sc-7e857967-1 exgdzz").find('div', class_="ipc-chip-list__scroller")

genres = []

for tag in req_div.find_all('a'):
    genres.append(tag.text)

print(genres)

['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western']


In [3]:
def write_to_json(new_data, filename):
    with open(filename, 'r+') as file:
        file_data = json.load(file)
        file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file, indent=4)

def time_delay():
    time.sleep(random.randint(2, 5))

In [4]:
movie_urls = set()
with open("top_20_movies.json", "w") as f:
    json.dump([], f)


for genre in genres:
    rank = genres.index(genre) + 1
    genre_ = genre.replace(' ', '-').lower()
    genre_url = 'https://www.imdb.com/search/title/?title_type=feature&genres=' + genre_
    response = requests.get(genre_url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    dom = et.HTML(str(soup))

    movie_names = dom.xpath('//div[@class="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-43986a27-9 gaoUku dli-title"]/a/h3/text()')
    movie_names = movie_names[:20]
    movie_names = [unidecode(i) for i in movie_names]
    movie_urls_list = dom.xpath('//div[@class="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-43986a27-9 gaoUku dli-title"]/a/@href')
    movie_urls_list = movie_urls_list[:20]

    for i in movie_urls_list:
        short_url = "https://www.imdb.com" + i.split("?")[0]
        movie_urls.add(short_url)
        # print(short_url)
    # print(movie_names)
    write_to_json({'s_no': rank, 'genre': genre, 'top_20': movie_names}, 'top_20_movies.json')

    # time_delay()
    print("{} written to json file\n".format(rank))
    


1 written to json file

2 written to json file

3 written to json file

4 written to json file

5 written to json file

6 written to json file

7 written to json file

8 written to json file

9 written to json file

10 written to json file

11 written to json file

12 written to json file

13 written to json file

14 written to json file

15 written to json file

16 written to json file

17 written to json file

18 written to json file

19 written to json file

20 written to json file

21 written to json file

22 written to json file

23 written to json file



In [5]:
print('total movies: ', len(movie_urls))

total movies:  264


In [7]:
def get_reviews(movie_url):
    review_url = "reviews?sort=submissionDate&dir=desc&ratingFilter=0"

    response = requests.get(movie_url + review_url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')

    review_container = soup.find('div', class_="lister-list")
    review_list = review_container.find_all('div', class_="lister-item-content")
    x = min(10, len(review_list))
    review_list = review_list[:x]

    reviews = []

    for review in review_list:
        review_title = review.find('a', class_="title").text.strip()
        review_text = review.find('div', class_="text show-more__control").text
        user_rating = review.find('span', class_="rating-other-user-rating").find('span').text + "/10"
        review_date = review.find('span', class_="review-date").text
        
        reviews.append({'review_title': review_title,
                        'review_text': review_text,
                        'user_rating': user_rating,
                        'review_date': review_date})

    return reviews

In [8]:
with open("movies_details.json", "w") as f:
    json.dump([], f)

for rank, movie_url in enumerate(movie_urls):
    response = requests.get(movie_url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    dom = et.HTML(str(soup))

    rank = rank + 1
    movie_name = dom.xpath('//span[@data-testid="hero__primary-text"]/text()')[0]
    movie_year = dom.xpath('//ul[@class="ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"]/li/a/text()')[0]
    director_name = dom.xpath('//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]/text()')[0]
    try:
        rating = dom.xpath('//span[@class="sc-bde20123-1 cMEQkK"]/text()')[0] + '/10'
        reviews = get_reviews(movie_url)
    except:
        rating = 'N/A'
        reviews = []
    actors_list = dom.xpath('//a[@data-testid="title-cast-item__actor"]/text()')
    actors_list = [unidecode(i) for i in actors_list]
    
    write_to_json({
                   's_no': rank,
                   'movie_title': movie_name,
                   'movie_year' : movie_year,
                   'director_name': unidecode(director_name),
                   'imdb_rating': rating,
                   'movie_url': movie_url,
                   'cast': actors_list,
                   'latest_10_reviews': reviews}, 'movies_details.json')

    time_delay()
    print("{} written to json file\n".format(rank))
    

    # to prevent running for long time since there are 200+ movies
    # comment this if you want to run for all movies
    if rank == 5:
        break
    
    

1 written to json file

2 written to json file

3 written to json file

4 written to json file

5 written to json file

